In [1]:
import pandas as pd
import re
from mlxtend.frequent_patterns import apriori, association_rules
data = pd.read_csv("Team_Sports_Survey_Preprocessed.csv")
# remove the eg. for each items
def remove_parentheses(text):
    if isinstance(text, str):  
        return re.sub(r"\s*\(.*?\)", "", text)
    return text  

In [2]:
# Get the rows which salary is below 9100
salary = data[data["Monthly Income (Php)"] == "< 9,100"]

In [3]:
# Clean the columns data by removing the (eg. items) by using the regex
columns_to_combine = ["Preferred Swimming Pool Activities", "Preferred Beach Activities", "Preferred Spa and Wellness Activities"]
salary[columns_to_combine] = salary[columns_to_combine].applymap(remove_parentheses)

# Combine all the columns
salary["Combined Activities"] = salary[columns_to_combine].apply(lambda x: ', '.join(x.dropna()), axis=1)

# Convert the items into a transactional data 
transactions = salary["Combined Activities"].str.split(",").explode().str.strip().reset_index()
transactions_encoded = transactions.pivot_table(index="index", columns="Combined Activities", aggfunc=lambda x: 1, fill_value=0)

C:\Users\joaqu\AppData\Local\Temp\ipykernel_19516\355822851.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  salary[columns_to_combine] = salary[columns_to_combine].applymap(remove_parentheses)
C:\Users\joaqu\AppData\Local\Temp\ipykernel_19516\355822851.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  salary[columns_to_combine] = salary[columns_to_combine].applymap(remove_parentheses)
C:\Users\joaqu\AppData\Local\Temp\ipykernel_19516\355822851.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

In [4]:
# Use Apriori algorithm to find frequent association
frequent_items= apriori(transactions_encoded, min_support=0.3, use_colnames=True)# threshold 30%
rule = association_rules(frequent_items)
rule[['antecedents','consequents','support', 'confidence', 'lift']].sort_values("confidence", ascending=False)

C:\Users\joaqu\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,support,confidence,lift
2,(Spa Treatment),(Massage Therapy),0.305085,1.000000,2.034483
6,(Spa Treatment),(Relaxing in Pool),0.305085,1.000000,1.311111
7,"(Massage Therapy, Spa Treatment)",(Relaxing in Pool),0.305085,1.000000,1.311111
8,"(Relaxing in Pool, Spa Treatment)",(Massage Therapy),0.305085,1.000000,2.034483
9,(Spa Treatment),"(Massage Therapy, Relaxing in Pool)",0.305085,1.000000,2.565217
3,(Play with Pool Toys),(Relaxing in Pool),0.322034,0.950000,1.245556
4,(Sauna and Steam Rooms),(Relaxing in Pool),0.338983,0.909091,1.191919
0,(Beach Swimming),(Relaxing in Pool),0.508475,0.857143,1.123810
5,(Snorkeling/Scuba Diving),(Relaxing in Pool),0.389831,0.851852,1.116872
1,(Kayaking/Paddling),(Relaxing in Pool),0.372881,0.814815,1.068313
